# 🧪 Step 1: Research & Data Modelling
**PR Branch Name:** banks-data-modelling

This notebook documents the process for Step 1 of the "Banks in Berlin" project:
- **1.1 Data Source Discovery**
- **1.2 Modelling & Planning**
- **1.3 Prepare the /sources Directory**
- **1.4 Review**

Goal:
- Identify and document relevant data sources.
- Select the 23 key parameters for our use case.
- Draft the planned table schema.
- Plan cleaning and transformation steps before database population.


## 1.1 Data Source Discovery

**Topic:** Banks in Berlin

**Main source:**
- **Name:** OpenStreetMap (OSM) via OSMnx library
- **Source and origin:** Public crowdsourced geospatial database
- **Update frequency:** Continuous (dynamic)
- **Data type:** Dynamic (API query using `amenity=bank`)
- **Reason for selection:**  
  - Covers all banks in Berlin  
  - Includes coordinates, names, addresses, and other useful attributes  
  - Open, free, and easy to query programmatically

**Optional additional sources:**
- **Name:** Berlin Open Data Portal (daten.berlin.de)
- **Source and origin:** Official Berlin city government
- **Update frequency:** Varies per dataset
- **Data type:** Static or semi-static (download as CSV/GeoJSON)
- **Possible usage:** Enrich with official administrative boundaries or extra metadata

**Enrichment potential:**
- Neighborhood/district info from Berlin shapefiles (GeoJSON)
- Linking to local amenities for spatial context


In [9]:
# Install Libraries

# %pip install osmnx geopandas pandas --quiet

In [10]:
# Import Libraries

import osmnx as ox
import geopandas as gpd
import pandas as pd

In [11]:
# Fetch banks in Berlin from OSM using the tag "amenity=bank"

tags = {"amenity": "bank"}

In [12]:
# Fetch geometries for Berlin

banks_gdf = ox.features_from_place("Berlin, Germany", tags)


In [13]:
# Display basic info

print(f"Number of bank entries fetched: {len(banks_gdf)}")
banks_gdf.head(3)

Number of bank entries fetched: 323


geometry addr:city addr:country  \
element id                                                           
node    28968292  POINT (13.31972 52.48667)    Berlin           DE   
        60848455  POINT (13.47104 52.53033)    Berlin          NaN   
        87040399   POINT (13.3888 52.51105)       NaN          NaN   

                 addr:housenumber addr:postcode          addr:street  \
element id                                                             
node    28968292               42         10713      Berliner Straße   
        60848455               13         10369  Anton-Saefkow-Platz   
        87040399              NaN           NaN                  NaN   

                  addr:suburb amenity  atm branch  ... operator:type  \
element id                                         ...                 
node    28968292  Wilmersdorf    bank  yes     eG  ...           NaN   
        60848455          NaN    bank  yes    NaN  ...           NaN   
        87040399          NaN    bank  NaN    NaN  ...           NaN   

                 start_date building:levels roof:levels roof:shape indoor  \
element id                                                                  
node    28968292        NaN             NaN         NaN        NaN    NaN   
        60848455        NaN             NaN         NaN        NaN    NaN   
        87040399        NaN             NaN         NaN        NaN    NaN   

                 access room western_union building:part  
element id                                                
node    28968292    NaN  NaN           NaN           NaN  
        60848455    NaN  NaN           NaN           NaN  
        87040399    NaN  NaN           NaN           NaN  

[3 rows x 100 columns]

In [14]:
banks_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
MultiIndex: 323 entries, ('node', np.int64(28968292)) to ('way', np.int64(611744021))
Data columns (total 100 columns):
 #   Column                                 Non-Null Count  Dtype   
---  ------                                 --------------  -----   
 0   geometry                               323 non-null    geometry
 1   addr:city                              227 non-null    object  
 2   addr:country                           141 non-null    object  
 3   addr:housenumber                       224 non-null    object  
 4   addr:postcode                          228 non-null    object  
 5   addr:street                            233 non-null    object  
 6   addr:suburb                            136 non-null    object  
 7   amenity                                323 non-null    object  
 8   atm                                    251 non-null    object  
 9   branch                                 9 non-null      object  
 10  b

## 1.2 Modelling & Planning

### Selected 23 Key Columns
1. osm_id
2. name
3. brand
4. operator
5. street
6. housenumber
7. postcode
8. city
9. country
10. phone
11. email
12. website
13. opening_hours
14. atm
15. wheelchair
16. building
17. latitude
18. longitude
19. geom_type
20. geom
21. neighbourhood
22. district
23. source

---

### How this connects to existing tables:
- **Coordinates (latitude, longitude, geom):** link to neighbourhood and district polygons.
- **Neighbourhood & district fields:** join with administrative boundaries table.
- **Source field:** ensures traceability.

---

### Planned Schema: `banks_in_berlin`
| Column Name     | Data Type | Description | Example |
|-----------------|-----------|-------------|---------|
| osm_id          | int       | Unique OSM ID | 12345678 |
| name            | text      | Bank name | Deutsche Bank |
| brand           | text      | Brand name if available | Sparkasse |
| operator        | text      | Entity operating the bank | Berliner Volksbank |
| street          | text      | Street name | Friedrichstraße |
| housenumber     | text      | House number | 45 |
| postcode        | text      | Postal code | 10117 |
| city            | text      | City name | Berlin |
| country         | text      | Country code | DE |
| phone           | text      | Contact phone | +49 30 123456 |
| email           | text      | Contact email | info@bank.de |
| website         | text      | Website URL | www.bank.de |
| opening_hours   | text      | Opening hours string | Mo-Fr 09:00-17:00 |
| atm             | text      | Presence of ATM | yes |
| wheelchair      | text      | Accessibility info | yes |
| building        | text      | Building type | yes |
| latitude        | float     | Latitude coordinate | 52.5200 |
| longitude       | float     | Longitude coordinate | 13.4050 |
| geom_type       | text      | Geometry type | Point |
| geom            | geometry  | Full geometry | (GeoJSON) |
| neighbourhood   | text      | Local neighbourhood name | Mitte |
| district        | text      | Berlin district | Mitte |
| source          | text      | Data source info | OSM |

---

### Known Data Issues
- Missing contact details for some entries.
- Inconsistent postcode and address formats.
- Neighbourhood and district not always included in raw OSM data.
- Opening hours in non-standard formats.

---

### Transformation Plan
1. Fetch data from OSM with filter `amenity=bank` (Berlin bounding box).
2. Clean column names → snake_case.
3. Normalize formats (phone, postcode, website URLs).
4. Enrich with neighbourhood/district via spatial join.
5. Save cleaned dataset (GeoJSON + CSV).


In [15]:
# Select 23 Columns & Add Coordinates

In [16]:
# Ensure geometry type is Point for lat/lon extraction

banks_gdf = banks_gdf.to_crs(epsg=4326)

In [ ]:
# Extract latitude and longitude

# banks_gdf["latitude"] = banks_gdf.geometry.y
# banks_gdf["longitude"] = banks_gdf.geometry.x

ValueError: y attribute access only provided for Point geometries

In [18]:
# Select the 23 columns (fill missing with None if not present)

selected_columns = [
    "osmid", "name", "brand", "operator",
    "addr:street", "addr:housenumber", "addr:postcode", "addr:city", "addr:country",
    "phone", "email", "website", "opening_hours",
    "atm", "wheelchair", "building",
    "latitude", "longitude", "geometry",
    # placeholders for enrichment
    "neighbourhood", "district",
    # add source info
    "source"
]

In [19]:
# Rename columns to match our schema

rename_map = {
    "osmid": "osm_id",
    "addr:street": "street",
    "addr:housenumber": "housenumber",
    "addr:postcode": "postcode",
    "addr:city": "city",
    "addr:country": "country"
}

In [20]:
banks_df = banks_gdf.rename(columns=rename_map)[[col if col in banks_gdf.columns else None for col in rename_map.values()]]

KeyError: "None of [Index([None, None, None, None, None, None], dtype='object')] are in the [columns]"

## 1.3 Prepare the /sources Directory

- **Raw Data Files:**  
    - `banks_raw.geojson` (includes geometry)  
    - `banks_raw.csv` (tabular only, no geometry)  

- **README.md** in `/sources` will contain:
    - Data sources used.
    - Planned transformation steps.


In [ ]:
# Save as GeoJSON (keeps geometry) and CSV

raw_geojson_path = "sources/banks_raw.geojson"
raw_csv_path = "sources/banks_raw.csv"


banks_gdf.to_file(raw_geojson_path, driver="GeoJSON")
banks_gdf.drop(columns="geometry").to_csv(raw_csv_path, index=False)

print(f"Raw data saved to: {raw_geojson_path} and {raw_csv_path}")

## 1.4 Review

- All 23 target columns defined.
- Data sources identified and documented.
- Schema draft created.
- Data fetched and stored in `/sources`.
- Data cleaning & enrichment plan in place.

**Next Step:** Step 2 — Fetch & Transform data.
